In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import os

### <a id='Introduction'><font color=black><u><center>Introduction - Python Web Scraping</center></u></font></a>
The python code below was used to scrape web pages from the job website, Indeed.com. 

The process was as follows:
1. Create a variable, URL, to hold a link to a search for Data Science job in New York, NY.
2. To navigate through each page of the search, the base url needs the extension, "&start=" with the next page beginning at 10.
3. Create a data frame to hold the results of the scrape.
4. Create a for loop which navigates through each page of the search for Data Science jobs in New York, clicks on each link for each job, scrapes the Job Title, Company, Description, and Link for each job listed.

In [ ]:
#Base Url for New York, NY Data Science Jobs
URL = "https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY"
Base_URL = "https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY"
pages = '&start='
counter = 9


In [ ]:
#Create the Python data frame to hold the results
JobsB_df = pd.DataFrame(columns=['Title', 'Company', 'Description', 'Link'])


In [ ]:
#For loop that navigates through the pages of the search result
for i in range(1,1000):
#Automates a FireFox web browser which searches and open links
    driver = webdriver.Firefox()
    print(URL)
    driver.get(URL)
    driver.implicitly_wait(20)
#Creates a list of links for each job listed on each page of the search.  Identifies the links by searching for XPath element for each job.
    elements = driver.find_elements_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "jobtitle", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "turnstileLink", " " ))]')
#Another for loop that loops through the elements list, clicks on each job link.   
    for job in elements:
        try:
            job.click()
#Scrapes the page by locating the XPath for the job description, title, company name and then appends results to the pandas data frame
            post = driver.find_element_by_xpath('//*[@id="vjs-desc"]').text
            title = driver.find_element_by_xpath('//*[@id="vjs-jobtitle"]').text 
            company = driver.find_element_by_xpath('//*[@id="vjs-cn"]/a').text             
            JobsB_df = JobsB_df.append({'Title':title,'Company':company, 'Description':post, 'Link':URL}, ignore_index=True)
        except:
            pass
#closes the browser        
    driver.quit()
#Increments the counter by one to move to the next page of the search and then appends to the base url which moves on to the next page.
    counter = counter + i
    pages = '&start=' + str(counter)
    URL = Base_URL + pages   
    counter = 9
    
    

In [ ]:
JobsB_df

In [ ]:
#Eliminates duplicate job postings
Final_Jobs_df=JobsB_df.drop_duplicates(['Company'], keep='first')

In [ ]:
#Writes the results to a csv file.
Final_Jobs_df.to_csv("Indeed Data Science Jobs Revised_01.csv")